In [46]:
import numpy as np
import cv2

In [47]:
# Paramètres globaux
N = 200  # Nombre de particules
sigma_position = np.diag([50, 50])  # Bruit gaussien pour la position
sigma_angle = 5  # Bruit pour l'angle en degrés
lambd = 5  # Paramètre de lissage pour la mise à jour des poids


In [48]:
# Chargement de la vidéo
cap = cv2.VideoCapture('video sequences/synthetic/escrime-4-3.avi')  # Exemple de chemin vidéo

# Lire la première image pour sélectionner la fenêtre
ret, frame = cap.read()
if not ret:
    print("Erreur : Impossible de lire la vidéo")
    cap.release()
    exit()

# Sélection de la région d'intérêt (ROI) pour l'objet
roi = cv2.selectROI("Select Object", frame, fromCenter=False, showCrosshair=True)
x, y, w, h = roi
cv2.destroyAllWindows()

In [49]:
# Calcul de l'histogramme de la fenêtre sélectionnée
def calc_histogram(image, roi):
    x, y, w, h = roi
    roi_image = image[int(y):int(y+h), int(x):int(x+w)]
    hist = cv2.calcHist([roi_image], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

roi_hist = calc_histogram(frame, roi)

In [50]:
# Initialisation des particules avec (x, y, theta)
particles = np.column_stack((
    np.random.uniform(x, x + w, N),        # Position x
    np.random.uniform(y, y + h, N),        # Position y
    np.random.uniform(-360, 360, N) # Angle theta en degrés
))
weights = np.ones(N) / N  # Poids initiaux uniformes

In [51]:
# Fonction de prédiction pour la position et l'angle
def predict(particles, sigma_position, sigma_angle):
    noise_position = np.random.multivariate_normal([0, 0], sigma_position, N)
    noise_angle = np.random.normal(0, sigma_angle, N) # Pour l'angle 
    particles[:, 0:2] += noise_position  # Mise à jour de x et y
    particles[:, 2] += noise_angle  # Mise à jour de theta
    particles[:, 2] %= 360  # Conserver l'angle dans [0, 360]
    return particles

In [52]:
# Calcul de l'histogramme en tenant compte de l'angle de rotation
def calc_histogram_with_orientation(image, particle, w, h):
    x, y, theta = particle
    M = cv2.getRotationMatrix2D((x, y), theta, 1.0)
    rotated_roi = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    roi = rotated_roi[int(y - h / 2):int(y + h / 2), int(x - w / 2):int(x + w / 2)]
    hist = cv2.calcHist([roi], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [53]:
# Mise à jour des poids en fonction de la similarité des histogrammes
def compute_weights(particles, frame, roi_hist, lambd):
    weights = np.zeros(N)
    for i, particle in enumerate(particles):
        patch_hist = calc_histogram_with_orientation(frame, particle, w, h)
        dist = cv2.compareHist(roi_hist, patch_hist, cv2.HISTCMP_BHATTACHARYYA)
        weights[i] = np.exp(-lambd * dist**2)
    weights /= np.sum(weights)  # Normalisation des poids
    return weights

In [54]:
# Rééchantillonnage des particules en fonction des poids
def resample(particles, weights):
    indices = np.random.choice(N, N, p=weights)
    particles = particles[indices]
    return particles

In [55]:
# Boucle principale de suivi avec affichage
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prédiction des particules (x, y, theta)
    particles = predict(particles, sigma_position, sigma_angle)

    # Correction et mise à jour des poids
    weights = compute_weights(particles, frame, roi_hist, lambd)

    # Rééchantillonnage
    particles = resample(particles, weights)

    # Estimation de la position et de l'angle en prenant la moyenne pondérée
    estimated_position = np.average(particles[:, :2], axis=0, weights=weights)
    estimated_angle = np.average(particles[:, 2], weights=weights)

    # Afficher le rectangle avec l'angle estimé
    oriented_rect = ((estimated_position[0], estimated_position[1]), (w, h), estimated_angle)
    box = cv2.boxPoints(oriented_rect)
    box = np.int0(box)
    cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)  # Dessiner le rectangle orienté

    # Afficher chaque particule en jaune
    for particle in particles:
        cv2.circle(frame, (int(particle[0]), int(particle[1])), 2, (0, 255, 255), -1)

    # Affichage de la frame avec le suivi en temps réel
    cv2.imshow("Tracking", frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Libération des ressources
cap.release()
cv2.destroyAllWindows()

/var/folders/vv/klkd4n6s4879bw6t07pfn6wm0000gp/T/ipykernel_13038/3701441691.py:23: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
